<a href="https://colab.research.google.com/github/abtraore/VGG16/blob/main/VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np
import cv2
layers = tf.keras.layers

In [ ]:
imagenette_map = { 
    "n01440764" : "tench",
    "n02102040" : "springer",
    "n02979186" : "casette_player",
    "n03000684" : "chain_saw",
    "n03028079" : "church",
    "n03394916" : "French_horn",
    "n03417042" : "garbage_truck",
    "n03425413" : "gas_pump",
    "n03445777" : "golf_ball",
    "n03888257" : "parachute"
}

In [ ]:
!wget  https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
!tar -xvf imagenette2.tgz

In [ ]:
# create a new generator
imagegen = ImageDataGenerator()
# load train data
train = imagegen.flow_from_directory("imagenette2/train/", class_mode="sparse", shuffle=True, batch_size=128, target_size=(224, 224))
# load val data
val = imagegen.flow_from_directory("imagenette2/val/", class_mode="sparse", shuffle=True, batch_size=128, target_size=(224, 224))

In [ ]:
class VGG(tf.keras.Model):

  def __init__(self):
    super(VGG, self).__init__()

    self.conv1_1 = layers.Conv2D(64,3,padding='same',activation="relu")
    self.conv1_2 = layers.Conv2D(64,3,padding='same',activation="relu")
    self.max_pool_1 = layers.MaxPool2D(2,strides=2)

    self.conv2_1 = layers.Conv2D(128,3,padding='same',activation="relu")    
    self.conv2_2 = layers.Conv2D(128,3,padding='same',activation="relu")
    self.max_pool_2 = layers.MaxPool2D(2)

    self.conv3_1 = layers.Conv2D(256,3,padding='same',activation="relu")    
    self.conv3_2 = layers.Conv2D(256,3,padding='same',activation="relu")
    self.conv3_3 = layers.Conv2D(256,3,padding='same',activation="relu")
    self.max_pool_3 = layers.MaxPool2D(2,strides=2)

    self.conv4_1 = layers.Conv2D(512,3,padding='same',activation="relu")    
    self.conv4_2 = layers.Conv2D(512,3,padding='same',activation="relu")
    self.conv4_3 = layers.Conv2D(512,3,padding='same',activation="relu")
    self.max_pool_4 = layers.MaxPool2D(2,strides=2)

    self.conv5_1 = layers.Conv2D(512,3,padding='same',activation="relu")    
    self.conv5_2 = layers.Conv2D(512,3,padding='same',activation="relu")
    self.conv5_3 = layers.Conv2D(512,3,padding='same',activation="relu")
    self.max_pool_5 = layers.MaxPool2D(2,strides=2)

    self.flatten = layers.Flatten()
    self.dropout = layers.Dropout(0.25)
    self.fully_connected_1 = layers.Dense(4096,activation="relu")
    self.fully_connected_2 = layers.Dense(4096,activation="relu")
    self.classification = layers.Dense(10,activation='softmax')

  def call(self,x):

    x = self.conv1_1(x)
    x = self.conv1_2(x)
    x = self.max_pool_1(x)

    x = self.conv2_1(x)
    x = self.conv2_2(x)
    x = self.max_pool_2(x)

    x = self.conv3_1(x)
    x = self.conv3_2(x)
    x = self.conv3_3(x)
    x = self.max_pool_3(x)

    x = self.conv4_1(x)
    x = self.conv4_2(x)
    x = self.conv4_3(x)
    x = self.max_pool_4(x)

    x = self.conv5_1(x)
    x = self.conv5_2(x)
    x = self.conv5_3(x)
    x = self.max_pool_5(x)

    x = self.flatten(x)
    x = self.dropout(x)
    x = self.fully_connected_1(x)
    x = self.fully_connected_2(x)
    x = self.classification(x)

    return x

In [ ]:
model = VGG()

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer   = tf.keras.optimizers.SGD()

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [ ]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
    
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [ ]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:
EPOCHS = 1000
ITER = 0
ITER_MAX = 10000

for epoch in range(EPOCHS):

  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train:
    train_step(images, labels)
    ITER += 256
    if ITER >= ITER_MAX:
        ITER = 0
        break 

  for test_images, test_labels in val:
    test_step(test_images, test_labels)
    ITER += 256
    if ITER >= ITER_MAX:
        ITER = 0
        break 

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )